In [1]:
import pickle
from functools import partial
from geopy.geocoders import Nominatim
import spacy

nlp = spacy.load("en_core_web_sm")

In [2]:
countries = ["Singapore", "UK", "US", "India"]

## Get results

The input is going to be the risk scores for the bilateral relations between all countries.

In [3]:
RelationsPred = []

for i in range(len(countries)):
    for j in range(len(countries) - i - 1):
        print(f"This is the relations between {countries[i]} and {countries[j + i + 1]}")
        RelationsPred.append("""The risk score""")

This is the relations between Singapore and UK
This is the relations between Singapore and US
This is the relations between Singapore and India
This is the relations between UK and US
This is the relations between UK and India
This is the relations between US and India


## Get actual

To do this look for:
- Get risk dimension: political or economic
- Articles talking about international events (Identify the countries involved)
- Extract the risk score

### Process articles

In [18]:
def InternationalDomestic(article):
    doc = nlp(article)

    places = []

    for ent in doc.ents:
        if ent.label_ == "GPE" :
            places.append(ent.text)

    geolocator = Nominatim(user_agent="diplomacy-software")

    geocode = partial(geolocator.geocode, language="en")

    CountriesFound = {}

    for place in places:
        try:
            country = geocode(place)[0].split(',')[-1].strip()

            if country not in CountriesFound:
                CountriesFound[country] = 1
            else:
                CountriesFound[country] += 1

        except TypeError:
            pass

    print(CountriesFound)

    total = sum(list(CountriesFound.values()))

    try:
        threshold = total / len(CountriesFound)
    except:
        print("This article doesn't ")
        return []

    relevant = []

    for country in CountriesFound:
        if CountriesFound[country] > threshold:
            relevant.append(country)

    print(relevant)

    return relevant

In [ ]:
def RiskDimension(title, article):
    pass

In [19]:
def ProcessArticle(title, article):
    print("Title : ", title)

    countriesFound = InternationalDomestic(article)
    
    if len(countriesFound) < 2:
        return None
    
    Dim = RiskDimension(title, article)
    
    if Dim != "politics" and Dim != "economics":
        return None
    
    # Get the risk score here

    # Write the required data into the database
    
    print("===================== Next Article =====================")

In [ ]:
with open("Articles.bin", "rb") as f:
    articles = pickle.load(f)

for title, article in articles.items():
    ProcessArticle(title, article)

## Comparing

In [26]:
RelationsPred = [1,2,3]
RelationsActual = [2,3,4]

In [27]:
def Magnitude(vector):
    sum = 0

    for i in vector:
        sum += i ** 2
    
    return sum ** 0.5

In [28]:
Error = [ abs(RelationsPred[i] - RelationsActual[i]) for i in range(len(RelationsActual)) ]
PercentError = ( Magnitude(Error) / Magnitude(RelationsActual) ) * 100
print(f"The prediction of the model is off by {PercentError}% from the acutal events")

The prediction of the model is off by 32.163376045133845% from the acutal events
